# Assignment 2

In [2]:
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Question 5

In [7]:
# Dimesions and resolution for the meshgrid 
r_max = 2
r_min = -2
res = 10**(-1)

x = np.arange(r_min, r_max, res)
y = np.arange(r_min, r_max, res)
z = np.arange(r_min, r_max, res)


GX, GY, GZ = np.meshgrid(x, y, z)

In [8]:
np.shape(GX)

(40, 40, 40)